# C2-C6

In [152]:
import pandas as pd
import os
import pathlib

def fill_quarts(
        data: pd.DataFrame,
        type_appareil: str,
) -> pd.DataFrame:
    if type_appareil == 'C2-C6':

        shift_data = data.shift(freq='15min')
        data = pd.concat([data, shift_data], axis=0)

    if type_appareil == 'C6-C20':
        shift_data = data.shift(freq='15min')
        data = pd.concat([data, shift_data], axis=0)
        for i in range(2):
            
            shift_data = shift_data.shift(freq='15min')

            data = pd.concat([data, shift_data], axis=0)

    return (data.sort_index())


month = '02'
type_appareil = [
    'C2-C6',
    'C6-C20',
]

file_directories = [f for f in pathlib.Path().glob(f'../test_data/{month}/*{type_appareil[0]}.Asc')]

C2C6_data = pd.DataFrame()
for file in file_directories:
    asc_data = pd.read_table(file)
    asc_data['Sampling date'] = pd.to_datetime(asc_data['Sampling date']).dt.round('15min')
    asc_data.set_index('Sampling date', inplace=True)
    C2C6_data = pd.concat([C2C6_data, asc_data])

C2C6_data.sort_index(inplace=True)

C2C6_data = fill_quarts(
    data=C2C6_data,
    type_appareil=type_appareil[0]
    )
C2C6_data = C2C6_data[~C2C6_data.index.duplicated(keep='first')]

# C6-C20

In [153]:
import pandas as pd
import os
import pathlib

type_appareil = [
    'C2-C6',
    'C6-C20',
]

file_directories = [f for f in pathlib.Path().glob(f'../test_data/{month}/*{type_appareil[1]}.Asc')]

C6C20_data = pd.DataFrame()
for file in file_directories:
    asc_data = pd.read_table(file)
    asc_data['Sampling date'] = pd.to_datetime(asc_data['Sampling date']).dt.round('15min')
    asc_data.set_index('Sampling date', inplace=True)
    if "CAL60" in str(file):
        asc_data = asc_data.shift(periods=-1, freq='15min')
    C6C20_data = pd.concat([C6C20_data, asc_data])

C6C20_data.sort_index(inplace=True)

C6C20_data = fill_quarts(
    data=C6C20_data,
    type_appareil=type_appareil[1]
    )
C6C20_data = C6C20_data[~C6C20_data.index.duplicated(keep='first')]

In [154]:
import datetime as dt
import calendar

def filter_month(
        data: pd.DataFrame,
        year: int,
        month: int,
        day: int = 1,
):
    start_date = dt.datetime(year, month, day)
    day_before_start_date = start_date - dt.timedelta(days=1)

    last_day_of_month = calendar.monthrange(year, month)[1]
    end_date = dt.datetime(
        year,
        month,
        last_day_of_month,
        )
    data = data[day_before_start_date:end_date]
    return (data)

C2C6_data = filter_month(data=C2C6_data, year=2022, month=2)
C6C20_data = filter_month(data=C6C20_data, year=2022, month=2)


TypeError: cannot do slice indexing on RangeIndex with these indexers [2022-01-31 00:00:00] of type datetime

In [148]:
(dt.datetime(
        2022,
        3,
        1,
        )).strftime(format="%Y-%m-%d %H:%M:%S")

'2022-03-01 00:00:00'

In [ ]:
xair_data = pd.concat([C2C6_data, C6C20_data], axis=1, join='inner').sort_index()
xair_data.to_csv('../output/test.csv')

In [131]:
xair_data

,Volume,ETHANE,ETHYLENE,PROPANE,PROPENE,I-BUTANE,N-BUTANE,ACETYLENE,TRANS-2-BUTENE,1-BUTENE,...,N-HEXADECANE,N-HEPTADECANE,PHENANTHRENE,N-OCTADECANE,ANTHRACENE,N-NONADECANE,N-EICOSANE,FLUORANTHENE,PYRENE,Unnamed: 63
Sampling date,,,,,,,,,,,,,,,,,,,,,
2022-02-01 09:30:00,101.16,1.76,0.86,0.30,0.00,0.16,0.35,0.07,0.03,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-01 09:45:00,101.22,1.93,0.81,0.85,0.00,0.16,0.34,0.00,0.06,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-01 10:00:00,101.22,1.93,0.81,0.85,0.00,0.16,0.34,0.00,0.06,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-01 10:15:00,101.19,1.72,0.79,0.27,0.00,0.14,0.36,0.00,0.07,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-01 10:30:00,101.19,1.72,0.79,0.27,0.00,0.14,0.36,0.00,0.07,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-27 23:00:00,102.30,6.04,9.60,8.96,0.61,0.99,2.67,0.22,0.44,0.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-27 23:15:00,102.30,6.04,9.60,8.96,0.61,0.99,2.67,0.22,0.44,0.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-02-27 23:30:00,102.40,5.21,9.45,4.50,0.54,0.83,2.47,0.00,0.21,0.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [12]:
import os

months = [int(x) for x in os.listdir("../test_data")]

In [19]:
if os.path.isdir("../test_data") is not True:
    print("break") 

break
